# Model demo



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from preprocess import *
from spacy.lang.en import English
import os
import numpy as np
from utils import save_file, load_file, load_model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
MODEL_FLAG = 'bilstm_max'
state_file_path = f'weights/{MODEL_FLAG}/{MODEL_FLAG}_best.pth'

labels = ['neutral', 'entailment', 'contradiction']
vocab_file = 'NLI_vocab.pkl'
folder = 'saved_files'
if os.path.exists(f'{folder}/{vocab_file}'):
    print('loading vocab from file')
    vocab = load_file(vocab_file)
else:
    print('creating vocab with training set')
    train_split = preprocess(split='train')
    vocab = create_vocab(train_split)
    save_file(vocab, vocab_file)

vocab_size = len(vocab.mapping)
embeddings = align_vocab_with_glove(vocab)
model = load_model(embeddings, labels, vocab_size, device, MODEL_FLAG, state_file_path)

loading vocab from file
saved_files/NLI_vocab.pkl
loaded embeddings from file
loading bilstm_max


In [4]:
def transform_sentence(sent, vocab):
    nlp = English()
    tokenizer = nlp.tokenizer
    sent = list(tokenizer(sent.lower()))
    sent_ids = []
    for token in sent:
        sent_ids.append(vocab.mapping.get(token.text, 0))
    return sent_ids, [len(sent_ids)]

transform_sentence('This maps sentences to id', vocab)

([29982, 17706, 25860, 30238, 14643, 7790], [6])

In [5]:
def make_prediction(sent1, sent2, vocab, model, model_flag):
    sent_ids1, length1 = transform_sentence(sent1, vocab)
    sent_ids2, length2 = transform_sentence(sent2, vocab)
    sent1 = torch.tensor([sent_ids1])
    sent2 = torch.tensor([sent_ids2])
    print(sent1.shape)
    if model_flag == 'bow':
        predictions = model(sent_ids1, length1, sent_ids2, length2)
    elif model_flag == 'lstm' or model_flag == 'bilstm' or model_flag == 'bilstm_max':
       
        predictions = model(sent1, length1, sent2, length2)
    predicted_label = torch.argmax(predictions, dim=1)
    print(predicted_label)
    labels = ['neutral', 'entailment', 'contradiction']
    print(predictions)
    return labels[predicted_label]

make_prediction('i like you', 'apple pie is nice', vocab, model, MODEL_FLAG)

torch.Size([1, 3])
tensor([2])
tensor([[9.5273e-11, 0.0000e+00, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)


'contradiction'